In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import resample
import matplotlib
matplotlib.use('Qt5Agg')  # or 'Qt5Agg' depending on your system

def plot_waveforms_from_folder_split(wav_folder, target_rate=250, max_duration_sec=12.0, num_plots=10):
    wav_files = sorted([f for f in os.listdir(wav_folder) if f.lower().endswith('.wav')])
    if not wav_files:
        print("No .wav files found.")
        return

    # Split files into 10 (approximately) equal chunks
    chunks = np.array_split(wav_files, num_plots)

    for i, chunk in enumerate(chunks):
        plt.figure(figsize=(12, 6))
        lines = []
        base_color = '#1f77b4'

        for filename in chunk:
            filepath = os.path.join(wav_folder, filename)
            try:
                sr, data = wavfile.read(filepath)

                if data.ndim == 2:  # stereo to mono
                    data = data.mean(axis=1)

                # Trim to max duration
                max_samples_orig = int(max_duration_sec * sr)
                if len(data) > max_samples_orig:
                    data = data[:max_samples_orig]

                # Resample to target_rate
                num_target_samples = int(len(data) * target_rate / sr)
                data_resampled = resample(data, num_target_samples)
                time = np.arange(len(data_resampled)) / target_rate

                line, = plt.plot(time, data_resampled, alpha=0.3, linewidth=0.8, color=base_color, label=filename)
                lines.append(line)

            except Exception as e:
                print(f"Error processing {filename}: {e}")

        plt.title(f'Waveforms (Plot {i+1} of {num_plots}) - Downsampled to {target_rate} Hz')
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.legend(loc='upper right', fontsize='small')
        plt.tight_layout()
        
        
        # Add cursor highlighting
        cursor = mplcursors.cursor(lines, hover=True)
    
        @cursor.connect("add")
        def on_add(sel):
            for line in lines:
                line.set_linewidth(0.8)
                line.set_alpha(0.3)
                line.set_color(line._color)
    
            sel.artist.set_linewidth(2.5)
            sel.artist.set_alpha(1.0)
            sel.artist.set_color('orange')
            sel.annotation.set_text(sel.artist._filename)
    
            def remove_highlight(*_):
                sel.artist.set_linewidth(0.8)
                sel.artist.set_alpha(0.3)
                sel.artist.set_color(sel.artist._color)
                sel.annotation.set_visible(False)
                plt.draw()
    
            sel.annotation.figure.canvas.mpl_connect('button_press_event', remove_highlight)

        plt.show()

if __name__ == "__main__":
    folder = r"C:\Users\Louis\PycharmProjects\HTS-Audio-Transformer\workspace\esc-50\raw\ESC-50-master\audio_gaussian"
    plot_waveforms_from_folder_split(folder)


NameError: name 'mplcursors' is not defined